### Training of a simple policy using the custom reward function

In [1]:
import sys

sys.path.append("../venv/")
!source ../venv/bin/activate

In [2]:
import numpy as np 
import pandas as pd

import math
import os

In [3]:
import gym
gym.__version__

'0.21.0'

In [4]:
from citylearn.citylearn import CityLearnEnv

In [5]:
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

In [6]:
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat

In [7]:
sys.path.append("../custom_reward")
from custom_reward import CustomReward

#### Create the environment

A function and a wrapper class as given in the local evaluation script provided by the challenge

In [8]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}

def create_citylearn_env(schema_path, reward_function, central_agent):
    env = CityLearnEnv(schema=schema_path, reward_function=reward_function, central_agent=central_agent)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env

create environment

In [9]:
schema_path = os.path.join("../data/", "schema_edited.json")
env, wrapper_env = create_citylearn_env(schema_path, CustomReward, True)


In [10]:
display(env.observation_names)
# schema_path = os.path.join("../data/", "schema_edited.json")
# env, wrapper_env = create_citylearn_env(schema_path, CustomReward, True)
# display(env.observation_names)

[['day_type',
  'hour',
  'outdoor_dry_bulb_temperature',
  'outdoor_dry_bulb_temperature_predicted_6h',
  'outdoor_dry_bulb_temperature_predicted_12h',
  'outdoor_dry_bulb_temperature_predicted_24h',
  'diffuse_solar_irradiance',
  'diffuse_solar_irradiance_predicted_6h',
  'diffuse_solar_irradiance_predicted_12h',
  'diffuse_solar_irradiance_predicted_24h',
  'direct_solar_irradiance',
  'direct_solar_irradiance_predicted_6h',
  'direct_solar_irradiance_predicted_12h',
  'direct_solar_irradiance_predicted_24h',
  'carbon_intensity',
  'indoor_dry_bulb_temperature',
  'non_shiftable_load',
  'solar_generation',
  'dhw_storage_soc',
  'electrical_storage_soc',
  'net_electricity_consumption',
  'electricity_pricing',
  'electricity_pricing_predicted_6h',
  'electricity_pricing_predicted_12h',
  'electricity_pricing_predicted_24h',
  'cooling_demand',
  'dhw_demand',
  'cooling_demand_without_control',
  'heating_demand_without_control',
  'non_shiftable_load_without_control',
  'dhw_de

In [11]:
# env.get_metadata()
# env.reward_function.env_metadata

Prepare for SB3

In [12]:
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)

Create SAC model

In [13]:
model = SAC("MlpPolicy", env, tensorboard_log="./tensorboard_logs/")

# model = SAC.load("models/custom_reward_SAC6.zip")
model.set_env(env)

# display(model.env.)

/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/hkolstee/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


/home/hkolstee/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Create custom callback to track reward values

In [14]:
class CustomCallback(BaseCallback):
    """
    Custom callback for plotting additional reward values in tensorboard
    """
    def __init__(self, verbose = 0):
        super().__init__(verbose)

    def _on_rollout_end(self) -> None:
        self.logger.record("Comfort", -self.training_env.get_attr("reward_function")[0].comfort[0])
        self.logger.record("Emissions", -self.training_env.get_attr("reward_function")[0].emissions[0])
        self.logger.record("Grid", -self.training_env.get_attr("reward_function")[0].grid[0])
        self.logger.record("Resilience", -self.training_env.get_attr("reward_function")[0].resilience[0])
        self.logger.record("unmet hours of thermal comfort (u)", -self.training_env.get_attr("reward_function")[0].u[0])
        self.logger.record("carbon emissions (g)", -self.training_env.get_attr("reward_function")[0].g[0])
        self.logger.record("ramping (r)", -self.training_env.get_attr("reward_function")[0].r[0])
        self.logger.record("daily peak (d)", -self.training_env.get_attr("reward_function")[0].d[0])
        self.logger.record("load factor (l)", -self.training_env.get_attr("reward_function")[0].l[0])
        self.logger.record("all-time peak (a)", -self.training_env.get_attr("reward_function")[0].a[0])
        self.logger.record("thermal resilience (m)", -self.training_env.get_attr("reward_function")[0].m[0])
        self.logger.record("normalized unserved energy (s)", -self.training_env.get_attr("reward_function")[0].s[0])

    def _on_step(self) -> bool:
        return super()._on_step()

Train

In [15]:
# model.learn(total_timesteps = env.get_metadata()["simulation_time_steps"] * 20, 
#             log_interval = 1)
model.learn(total_timesteps = env.get_metadata()["simulation_time_steps"] * 35, 
            log_interval = 1,
            callback = CustomCallback()
            )
# model.save("models/custom_reward_SAC")

/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_function to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_function` for environment variables or `env.get_wrapper_attr('reward_function')` that will search the reminding wrappers.
  logger.warn(


Evaluate 

In [16]:
observations = env.reset()

while not env.done:
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, _, _ = env.step(actions)

kpis = env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value')
kpis = kpis.dropna(how='all')
display(kpis)

name,Building_1,Building_2,Building_3,District
cost_function,,,,
annual_normalized_unserved_energy_total,0.013111,0.015520,0.014719,0.014450
annual_peak_average,NaN,NaN,NaN,1.169707
carbon_emissions_total,1.800413,1.550902,1.327710,1.559675
cost_total,1.743678,1.516441,1.307621,1.522580
daily_one_minus_load_factor_average,NaN,NaN,NaN,0.833376
daily_peak_average,NaN,NaN,NaN,1.258661
discomfort_delta_average,-8.692588,-2.738613,-2.157545,-4.529582
discomfort_delta_maximum,2.087805,5.446190,4.122473,3.885489
discomfort_delta_minimum,-14.988202,-10.706911,-6.385336,-10.693483
